# PBP Cleaner for ISFL

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
from urllib.request import urlopen
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

import warnings
warnings.filterwarnings('ignore')

/Users/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  del sys.path[0]


In [2]:
pbpDF = pd.read_csv('allPBP.csv')

In [3]:
pbpDF.loc[pbpDF['play'].str.contains('FUMBLE')].head(15)

,Unnamed: 0,index,Unnamed: 0.1,teamID,Q,time,totTime,down,distance,side,yard,play,gameID,S,homeTeam,awayTeam,teamPoss,dist2goal,awayScore,homeScore
318,318,318,130,1,3,2:34,1054,3.0,2.0,COL,47.0,"Pass by Kyubee, S., complete to Lecavalier, R. for 11 yds. Tackle by Marmeladov, V.. FUMBLE by Lecavalier, R., recovered by Lecavalier, R..",18,1,COL,BAL,BAL,47.0,13,15
491,491,491,125,5,3,5:18,1218,1.0,10.0,OCO,33.0,"Boss, M. SACKED by Blaster Blade - DE for -11 yds. FUMBLE by Boss, M., recovered by Boss, M..",19,1,OCO,ARI,OCO,67.0,17,6
576,576,576,20,1,1,8:51,3231,3.0,12.0,BAL,37.0,"Rush by Robinson, M. for 6 yds. Tackle by Bavitz, I.. FUMBLE by Robinson, M., recovered by Novel, L..",20,1,BAL,OCO,BAL,63.0,0,7
741,741,741,3,2,1,14:30,3570,2.0,1.0,YKW,22.0,"Orosz, C. SACKED by Everest Teagarden - DE for -5 yds. FUMBLE by Orosz, C., recovered by Akselsen, M..",21,1,COL,YKW,YKW,78.0,0,0
768,768,768,30,3,1,5:44,3044,3.0,19.0,COL,22.0,"Rush by Tweed, B. for 8 yds. Tackle by Shelton, L.. FUMBLE recovered by Shelton, L. at the COL - 30 yard line and returned for 4 yards.",21,1,COL,YKW,COL,78.0,0,3
1170,1170,1170,6,4,1,13:36,3516,3.0,24.0,ARI,16.0,"Rush by Mackworthy, R. for 8 yds. Tackle by Showcase, S.. FUMBLE by Mackworthy, R., recovered by Mackworthy, R..",35,1,BAL,ARI,ARI,84.0,0,0
1397,1397,1397,24,2,1,6:23,3083,2.0,2.0,COL,25.0,"Pass by Orosz, C., complete to LeClair, A. for 22 yds. Tackle by Hayden, A.. FUMBLE by LeClair, A., recovered by LeClair, A..",36,1,YKW,COL,YKW,25.0,0,0
1478,1478,1478,105,2,3,14:35,1775,2.0,10.0,YKW,13.0,"Rush by Stats, J. for 0 yds. Tackle by Cox, V.. FUMBLE by Stats, J., recovered by Larsendorf, R..",36,1,YKW,COL,YKW,87.0,7,16
1573,1573,1573,4,6,1,13:54,3534,2.0,6.0,SJS,9.0,"Pass by Christ, J., complete to Bottles, B. for 11 yds. Tackle by Bavitz, I.. FUMBLE recovered by Bavitz, I. at the SJS - 20 yard line and returned for 2 yards.",37,1,OCO,SJS,SJS,91.0,0,0
2111,2111,2111,137,4,3,8:48,1428,1.0,10.0,ARI,28.0,"Pass by Draxel, V., complete to Jefferson, L. for 10 yds. Tackle by Shelton, L.. FUMBLE recovered by Sanchez, P. at the ARI - 38 yard line and returned for 1 yards.",42,1,ARI,YKW,ARI,72.0,13,7


In [4]:
testDF = pbpDF.loc[pbpDF['gameID'] == 5381]

In [5]:
def playType(x):
    play = x.loc['play']
    if 'Rush' in play:
        return 'Rush'
    elif 'Pass' in play or 'throw' in play or 'SACKED' in play:
        return 'Pass'
    elif 'Penalty' in play:
        return 'Penalty'
    elif 'Punt' in play:
        return 'Punt'
    elif 'kicks off' in play or 'Kickoff' in play:
        return 'Kickoff'
    elif 'FG' in play:
        return 'Field Goal'
    elif 'Timeout' in play:
        return 'Timeout'
    elif 'TOUCHDOWN' in play:
        return 'PAT'
    elif 'Returned by' in play:
        return 'Punt'
    elif 'takes it down' in play or 'waits for blockers' in play or 'endzone' in play or 'return' in play or 'cuts across' in play:
        return 'Kickoff'
    elif 'Turnover' in play:
        return 'Turnover on Downs'
    else:
        return 'N/A'

In [6]:
tqdm.pandas(desc="Play type")
testDF.loc[:,'playType'] = testDF.progress_apply(lambda x: playType(x),axis=1)

### Passing Columns

Offense:
`passResult`, `passer`, `receiver`, `incompleteType`

Defense:
`interceptor`, `passDefender`, `sacker`

In [7]:
def passResult(x):
    play = x.loc['play']
    playType = x.loc['playType']
    
    if playType == 'Pass':
        if 'incomplete' in play or 'Incomplete' in play or 'throws the ball away' in play:
            return 'Incomplete'
        elif 'complete' in play: return 'Complete'
        elif 'SACKED' in play: return 'Sack'
        elif 'INTERCEPT' in play: return 'Interception'
        else: return 'None'        
    else: return 'N/A'
    
def incompleteType(x):
    play = x.loc['play']
    
    if 'falls incomplete' in play:
        return 'Miss'
    elif 'Broken up' in play:
        return 'Broken up'
    elif 'dropped' in play:
        return 'Dropped'
    elif 'throws the ball away' in play:
        return 'Throw away'
    else:
        return 'N/A'

def passer(x):
    play = x.loc['play']
    passResult = x.loc['passResult']
    incompleteType = x.loc['incompleteType']
    
    if passResult == 'Sack':
        passer = play.split(' SACKED')[0]
    elif passResult == 'Complete':
        passer = play.split('Pass by ')[1].split(', c')[0]
    elif passResult == 'Interception':
        passer = play.split('Pass by ')[1].split(', t')[0]
    elif passResult == 'Incomplete':
        if incompleteType == 'Throw away':
            passer = play.split(' throws')[0]
        else:
            passer = play.split('Pass by ')[1].split(' to')[0]
    else:
        return 'N/A'
    return passer

def receiver(x):
    play = x.loc['play']
    passResult = x.loc['passResult']
    incompleteType = x.loc['incompleteType']
    
    if passResult == 'Incomplete':
        if incompleteType == 'Miss':
            receiver = play.split('to ')[1].split(' falls')[0]
            return receiver
        elif incompleteType == 'Broken up':
            receiver = play.split('to ')[1].split(' is incomplete')[0]
            return receiver
        elif incompleteType == 'Dropped':
            receiver = play.split('to ')[1].split(' was dropped')[0]
            return receiver
        else: return "N/A"
    elif passResult == 'Complete':
        receiver = play.split('complete to ')[1].split(' for')[0]
        return receiver
    elif passResult == 'Interception':
        receiver = play.split('to ')[1].split('. INTERCEPTION')[0]
        return receiver
    else: return "N/A"

In [8]:
tqdm.pandas(desc="Pass type")
testDF.loc[:,'passResult'] = testDF.progress_apply(lambda x: passResult(x),axis=1)

tqdm.pandas(desc="Incomplete type")
testDF.loc[:,'incompleteType'] = testDF.progress_apply(lambda x: incompleteType(x),axis=1)

tqdm.pandas(desc="Passer")
testDF.loc[:,'passer'] = testDF.progress_apply(lambda x: passer(x),axis=1)

tqdm.pandas(desc="Receiver")
testDF.loc[:,'receiver'] = testDF.progress_apply(lambda x: receiver(x),axis=1)

### Rushing Columns

Offense:
`rusher`, `rushResult`

In [9]:
def rushResult(x):
    play = x.loc['play']
    playType = x.loc['playType']
    
    if playType == 'Rush':
        if 'FUMBLE' in play:
            return 'Fumble'
        else:
            return 'Rush'
    else: return 'N/A'
    
def rusher(x):
    play = x.loc['play']
    playType = x.loc['playType']
    
    if playType == 'Rush':
        rusher = play.split('by ')[1].split(' for')[0]
    else:
        return 'N/A'

    return rusher

In [10]:
tqdm.pandas(desc="Rush type")
testDF.loc[:,'rushResult'] = testDF.progress_apply(lambda x: rushResult(x),axis=1)

tqdm.pandas(desc="rusher")
testDF.loc[:,'rusher'] = testDF.progress_apply(lambda x: rusher(x),axis=1)

In [11]:
# testDF.loc[testDF['playType'] == 'Rush']
testDF.columns

Index(['Unnamed: 0', 'index', 'Unnamed: 0.1', 'teamID', 'Q', 'time', 'totTime',
       'down', 'distance', 'side', 'yard', 'play', 'gameID', 'S', 'homeTeam',
       'awayTeam', 'teamPoss', 'dist2goal', 'awayScore', 'homeScore',
       'playType', 'passResult', 'incompleteType', 'passer', 'receiver',
       'rushResult', 'rusher'],
      dtype='object')

### To-Do:
* ~~Passing~~
* ~~Receiving~~
* ~~Rushing~~
* Defense (tackler, fumbler, fumble recoverer)
* Field Goals (kickdistance, result, blocker)
* Punts (kickdistance, result, yards returned, blocker)
* Extra Points (result, blocker)
* Play Result (TD/FD/ydGained)
* Kickoffs (kickdistance, result, yards returned)
* Dummies using `pd.get_dummies()`

In [12]:
pd.get_dummies(testDF['passResult'])

,Complete,Incomplete,Interception,N/A,Sack
282569,0,0,0,1,0
282570,0,0,0,1,0
282571,0,0,0,1,0
282572,0,0,0,1,0
282573,0,0,0,1,0
282574,0,0,0,1,0
282575,0,0,0,1,0
282576,0,0,0,0,1
282577,0,1,0,0,0
282578,0,0,0,1,0


In [13]:
testDF[testDF['playType'] == 'Punt']

,Unnamed: 0,index,Unnamed: 0.1,teamID,Q,time,totTime,down,distance,side,yard,play,gameID,S,homeTeam,awayTeam,teamPoss,dist2goal,awayScore,homeScore,playType,passResult,incompleteType,passer,receiver,rushResult,rusher
282578,282578,4504,9,4,1,12:58,3478,4.0,12.0,HON,47.0,"Punt by Powers, V. of 52 yards. Touchback.",5381,22,AZ,HON,HON,53.0,0,0,Punt,N/A,N/A,N/A,N/A,N/A,N/A
282610,282610,4536,41,12,1,0:38,2738,4.0,9.0,AZ,45.0,"Punt by Jay-Jaymison, J. of 55 yards. Touchback.",5381,22,AZ,HON,AZ,55.0,7,7,Punt,N/A,N/A,N/A,N/A,N/A,N/A
282616,282616,4542,47,4,2,14:55,2695,4.0,8.0,HON,33.0,"Punt by Powers, V. of 49 yards. No return.",5381,22,AZ,HON,HON,67.0,7,7,Punt,N/A,N/A,N/A,N/A,N/A,N/A
282672,282672,4598,103,12,3,13:35,1715,4.0,7.0,AZ,18.0,"Punt by Jay-Jaymison, J. of 47 yards.",5381,22,AZ,HON,AZ,82.0,14,13,Punt,N/A,N/A,N/A,N/A,N/A,N/A
282673,282673,4599,104,12,3,13:35,1715,NaN,NaN,NaN,NaN,"Returned by Kane-Maika'i (R), K. for 3 yards.",5381,22,AZ,HON,HON,NaN,14,13,Punt,N/A,N/A,N/A,N/A,N/A,N/A
282679,282679,4605,110,4,3,11:15,1575,4.0,8.0,AZ,33.0,"Punt by Powers, V. of 33 yards. Touchback.",5381,22,AZ,HON,HON,33.0,14,13,Punt,N/A,N/A,N/A,N/A,N/A,N/A
282686,282686,4612,117,4,3,8:48,1428,4.0,15.0,AZ,48.0,"Punt by Powers, V. of 31 yards. No return.",5381,22,AZ,HON,HON,48.0,14,13,Punt,N/A,N/A,N/A,N/A,N/A,N/A
282702,282702,4628,133,4,3,2:36,1056,4.0,13.0,HON,28.0,"Punt by Powers, V. of 51 yards.",5381,22,AZ,HON,HON,72.0,14,16,Punt,N/A,N/A,N/A,N/A,N/A,N/A
282703,282703,4629,134,4,3,2:36,1056,NaN,NaN,NaN,NaN,"Returned by Scarlett, D. for 18 yards.",5381,22,AZ,HON,AZ,NaN,14,16,Punt,N/A,N/A,N/A,N/A,N/A,N/A
282731,282731,4657,162,4,4,9:58,598,4.0,10.0,AZ,46.0,"Punt by Powers, V. of 35 yards.",5381,22,AZ,HON,HON,46.0,14,23,Punt,N/A,N/A,N/A,N/A,N/A,N/A
